In [106]:
from math import ceil
import pandas as pd

def min_count(W, C):
    return ceil(sum(W) / C)

def NFA(W, C, sort=False):
    NFA.comparisons = 0
    def _key_sort(x):
        NFA.comparisons += 1
        return x
    if sort:
        W = sorted(W, key=_key_sort, reverse=True)
    NFA.history = [dict(
        weight_index=1,
        weight=W[0],
        container_index=1,
    )]
    W = W[:]
    Cs = W[:1]
    for i, w in enumerate(W[1:]):
        NFA.comparisons += 1
        if w < C - Cs[-1]:
            Cs[-1] += w
        else:
            Cs.append(w)
        NFA.history.append(dict(
            weight_index=i + 2,
            weight=w,
            container_index=len(Cs),
        ))
    return len(Cs), NFA.comparisons, pd.pivot_table(pd.DataFrame(NFA.history), index='container_index', columns='weight_index', values='weight').fillna('')

def FFA(W, C, sort=False):
    FFA.comparisons = 0
    def _key_sort(x):
        FFA.comparisons += 1
        return x
    if sort:
        W = sorted(W, key=_key_sort, reverse=True)
    FFA.history = [dict(
        weight_index=1,
        weight=W[0],
        container_index=1,
    )]
    W = W[:]
    Cs = W[:1]
    for j, w in enumerate(W[1:]):
        b = True
        for i in range(len(Cs)):
            if not b:
                continue
            FFA.comparisons += 1
            if w < C - Cs[i] and b:
                Cs[i] += w
                b = False
        if b:
            Cs.append(w)
        FFA.history.append(dict(
            weight_index=j + 2,
            weight=w,
            container_index=i + 1,
        ))
    return len(Cs), FFA.comparisons, pd.pivot_table(pd.DataFrame(FFA.history), index='container_index', columns='weight_index', values='weight').fillna('')

def WFA(W, C, sort=False):
    WFA.comparisons = 0
    def _key_sort(x):
        WFA.comparisons += 1
        return x
    if sort:
        W = sorted(W, key=_key_sort, reverse=True)
    WFA.history = [dict(
        weight_index=1,
        weight=W[0],
        container_index=1,
    )]
    W = W[:]
    Cs = W[:1]
    for i, w in enumerate(W[1:]):
        worst = Cs.index(min(Cs))
        WFA.comparisons += (len(Cs) + worst + 1)
        if w < C - Cs[worst]:
            Cs[worst] += w
        else:
            worst = len(Cs)
            Cs.append(w)
        WFA.history.append(dict(
            weight_index=i + 2,
            weight=w,
            container_index=worst + 1,
        ))
    return len(Cs), WFA.comparisons, pd.pivot_table(pd.DataFrame(WFA.history), index='container_index', columns='weight_index', values='weight').fillna('')

def BFA(W, C, sort=False):
    BFA.comparisons = 0
    def _key_sort(x):
        BFA.comparisons += 1
        return x
    if sort:
        W = sorted(W, key=_key_sort, reverse=True)
    BFA.history = [dict(
        weight_index=1,
        weight=W[0],
        container_index=1,
    )]
    W = W[:]
    Cs = W[:1]
    def _greater(a, b):
        BFA.comparisons += 1
        return a > b
    for i, w in enumerate(W[1:]):
        prepared = [c if _greater(C - c, w) else -1 for c in Cs]
        worst = prepared.index(max(prepared))
        BFA.comparisons += 1 + worst
        if prepared[worst] == -1:
            worst = len(Cs)
            Cs.append(w)
        else:
            Cs[worst] += w
        BFA.history.append(dict(
            weight_index=i + 2,
            weight=w,
            container_index=worst + 1,
        ))
    return len(Cs), BFA.comparisons, pd.pivot_table(pd.DataFrame(BFA.history), index='container_index', columns='weight_index', values='weight').fillna('')




Ws = '47 24 82 35 32 04 54 43 98 86 40 78 59 62 62 83 41 48 23 24\n72 22 54 35 21 57 65 47 71 76 69 18 01 03 53 33 07 59 28 06\n97 20 84 08 34 98 91 76 98 15 52 71 89 59 06 10 16 24 09 39'
Ws = [list(map(int, w.split(' '))) for w in Ws.split('\n')]
C = 100

res_count = list()
res_diff = list()

for i, W in enumerate(Ws + [Ws[0] + Ws[1] + Ws[2]]):
    row1 = dict()
    row2 = dict()
    
    row1['Аналитический расчет'] = min_count(W, C)
    
    row1['NFA'] = NFA(W, C)[0]
    row1['FFA'] = FFA(W, C)[0]
    row1['WFA'] = WFA(W, C)[0]
    row1['BFA'] = BFA(W, C)[0]
    row2['BFA']  = BFA(W, C)[1]
    row2['NFA']  = NFA(W, C)[1]
    row2['FFA']  = FFA(W, C)[1]
    row2['WFA']  = WFA(W, C)[1]
    
    row1['NFA sorted'] = NFA(W, C, True)[0]
    row1['FFA sorted'] = FFA(W, C, True)[0]
    row1['WFA sorted'] = WFA(W, C, True)[0]
    row1['BFA sorted'] = BFA(W, C, True)[0]
    row2['FFA sorted']  = FFA(W, C, True)[1]
    row2['WFA sorted']  = WFA(W, C, True)[1]
    row2['NFA sorted']  = NFA(W, C, True)[1]
    row2['BFA sorted']  = BFA(W, C, True)[1]
    
    res_count.append(row1)
    res_diff.append(row2)

In [100]:
print('NFA')
NFA(Ws[0], C)[2]

NFA


weight_index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
container_index,,,,,,,,,,,,,,,,,,,,
1,47,24,,,,,,,,,,,,,,,,,,
2,,,82,,,,,,,,,,,,,,,,,
3,,,,35,32,4,,,,,,,,,,,,,,
4,,,,,,,54,43,,,,,,,,,,,,
5,,,,,,,,,98,,,,,,,,,,,
6,,,,,,,,,,86,,,,,,,,,,
7,,,,,,,,,,,40,,,,,,,,,
8,,,,,,,,,,,,78,,,,,,,,
9,,,,,,,,,,,,,59,,,,,,,


In [101]:
print('FFA')
FFA(Ws[0], C)[2]

FFA


weight_index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
container_index,,,,,,,,,,,,,,,,,,,,
1,47,24,82,,,,,,,,,,,,,,,,,
2,,,,35,,,,,,,,,,,,,,,,
3,,,,,32,4,54,,,,,,,,,,,,,
4,,,,,,,,43,98,,,,,,,,,,,
5,,,,,,,,,,86,,,,,,,,,,
6,,,,,,,,,,,40,,,,,,,,,
7,,,,,,,,,,,,78,,,,,,,,
8,,,,,,,,,,,,,59,62,,,,,,
9,,,,,,,,,,,,,,,62,,,,,


In [102]:
print('WFA')
WFA(Ws[0], C)[2]

WFA


weight_index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
container_index,,,,,,,,,,,,,,,,,,,,
1,47,24,,,,,,,,,,,,,,,,,,
2,,,82,,,,,,,,,,,,,,,,,
3,,,,35,32,4,,,,,,,,,,,,,,
4,,,,,,,54,43,,,,,,,,,,,,
5,,,,,,,,,98,,,,,,,,,,,
6,,,,,,,,,,86,,,,,,,,,,
7,,,,,,,,,,,40,,59,,,,,,,
8,,,,,,,,,,,,78,,,,,,,,
9,,,,,,,,,,,,,,62,,,,,23,


In [103]:
print('BFA')
BFA(Ws[0], C)[2]

BFA


weight_index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
container_index,,,,,,,,,,,,,,,,,,,,
1,47,24,,,,,,,,,,,,,,,,,23,
2,,,82,,,4,,,,,,,,,,,,,,
3,,,,35,32,,,,,,,,,,,,,,,24
4,,,,,,,54,43,,,,,,,,,,,,
5,,,,,,,,,98,,,,,,,,,,,
6,,,,,,,,,,86,,,,,,,,,,
7,,,,,,,,,,,40,,59,,,,,,,
8,,,,,,,,,,,,78,,,,,,,,
9,,,,,,,,,,,,,,62,,,,,,


In [108]:
print('Количество контейнеров:')
pd.DataFrame(res_count)

Количество контейнеров:


,BFA,BFA sorted,FFA,FFA sorted,NFA,NFA sorted,WFA,WFA sorted,Аналитический расчет
0,12,12,12,12,14,14,12,12,11
1,10,10,10,10,10,12,10,10,8
2,11,11,11,11,13,12,12,11,10
3,31,29,32,29,37,37,34,30,29


In [107]:
print('Сложность:')
pd.DataFrame(res_diff)

Сложность:


,BFA,BFA sorted,FFA,FFA sorted,NFA,NFA sorted,WFA,WFA sorted
0,165,256,98,150,19,39,211,306
1,154,206,72,97,19,39,195,266
2,199,250,93,145,19,39,236,309
3,1398,1924,734,963,59,119,1874,2406
